In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, display

plt.ioff()

In [ ]:
def ofc_update(avalanche,ofc_state,avhistory,t):
    retain = 0.25
    if not avalanche:
        kmax = np.amax(ofc_state)
        ofc_state += (1 -kmax)
        t += (1-kmax)
        avalanche = True
        avhistory = np.zeros(ofc_state.shape)
        return avalanche, ofc_state, avhistory,t
    if avalanche:
        mask_state = ofc_state * (ofc_state >= 1)
        avhistory = avhistory + (ofc_state >= 1)
        roll1 = np.pad(mask_state[1:,:],[(0,1),(0,0)],'constant')
        roll2 = np.pad(mask_state[:-1,:],[(1,0),(0,0)],'constant') 
        roll3 = np.pad(mask_state[:,1:],[(0,0),(0,1)],'constant') 
        roll4 = np.pad(mask_state[:,:-1],[(0,0),(1,0)],'constant')
        critsites = retain*(roll1+roll2+roll3+roll4)
        ofc_state[ofc_state>=1]=0
        ofc_state += critsites
        if np.amax(ofc_state) < 1:
            avalanche = False
        return avalanche, ofc_state, avhistory,t

In [ ]:
L = 20
ofc_state = np.random.rand(L,L)
avhistory = np.zeros(ofc_state.shape)
avalanche = False
ofc_avmags = np.zeros(500)
aftershocks_t=[0]
aftershocks_m=[0]
t=0

plt.close()
fig = plt.figure()
plt.subplot2grid((2,2),(0,0))
im1=plt.imshow(ofc_state, animated=True, cmap='plasma', interpolation='none')
plt.subplot2grid((2,2),(0,1))
im2=plt.imshow(ofc_state, animated=True, cmap='plasma', interpolation='none', vmax=4)
ax1 = plt.subplot2grid((2,2),(1,0))
plot1, = plt.plot([],[],'bx ')
ax1.set_xlim(0,2)
ax1.set_ylim(0,15)
ax2 = plt.subplot2grid((2,2),(1,1))
plot2, = plt.plot(np.random.rand(500),'bx ')

def updatefig(i):
    global avalanche
    global ofc_state
    global avhistory
    global avmags
    global aftershocks
    global t
    avalanche,ofc_state,avhistory,t = ofc_update(avalanche,ofc_state,avhistory,t)
    if not avalanche:
        avmag = int(np.sum(avhistory))
        ofc_avmags[avmag] += 1
        aftershocks_t.append(t)
        aftershocks_m.append(avmag)
    im1.set_array(ofc_state)
    im2.set_array(avhistory)  
    plot1.set_data(aftershocks_t[-10:],aftershocks_m[-10:])
    ax1.set_ylim([0,np.amax(aftershocks_m[-10:])+1])
    ax1.set_xlim([np.amin(aftershocks_t[-10:])-0.001,np.amax(aftershocks_t[-10:])+0.001])
    plot2.set_ydata(ofc_avmags)
    ax2.set_ylim([0,np.amax(ofc_avmags)+1])
    return (im1,im2,plot1,ax1,plot2,ax2,)

In [ ]:
anim = animation.FuncAnimation(fig, updatefig, frames='none', interval=100)
plt.show()

In [ ]:
def bs_update(avalanche,bs_state,threshold,avlength):
    avlength+=1
    fmin = np.amin(bs_state)
    i, = np.where(bs_state == fmin)
    changed=[(i[0]+1)%L,i[0],i[0]-1]
    bs_state[changed] = np.random.normal()
    if np.amin(bs_state) < threshold:
        avalanche = True
        return avalanche,bs_state,threshold,changed,avlength
    else:
        threshold = np.amin(bs_state)
        avalanche = False
        return avalanche,bs_state,threshold,changed,avlength

In [ ]:
L = 200
bs_state = np.random.rand(L)
threshold = 0
avalanche = False
bs_avmags = np.zeros(500)
avlenght = 0
pophistory = np.zeros((1000,L))

plt.close()
fig = plt.figure()
plt.subplot2grid((2,2),(0,0))
line,=plt.plot(bs_state,'bx ')
line2 = plt.axhline(threshold,color='r')
plt.subplot2grid((2,2),(0,1))
im2=plt.imshow(np.random.rand(1000,L), animated='True', cmap='plasma_r', interpolation='bilinear',aspect='auto',vmax=1000)
ax1 = plt.subplot2grid((2,2),(1,0),colspan=2)
plot1, = plt.plot(np.random.rand(500),'bx ')

def updatefig(i):
    global avalanche
    global bs_state
    global threshold
    global avlength
    global avmags
    global pophistory
    avalanche,bs_state,threshold,changed,avlength = bs_update(avalanche,bs_state,threshold,avlength)
    pophistory[:-1,:] = pophistory[1:,:]
    pophistory[-1,:] = pophistory[-1,:] + 1
    pophistory[-1,changed] = 0
    if not avalanche:
        bs_avmags[avlength] += 1
        avlength = 0
    line.set_ydata(bs_state)
    line2.set_ydata(threshold)
    im2.set_array(pophistory)
    plot1.set_ydata(bs_avmags)
    ax1.set_ylim([0,np.amax(bs_avmags)+1])
    return (line,line2,im2,ax1,plot1,)

In [ ]:
anim = animation.FuncAnimation(fig, updatefig, frames='none', interval=100)
plt.show()

In [ ]:
fig = plt.figure()
plt.subplot2grid((1,2),(0,0))
plt.loglog(ofc_avmags,'bx ')
plt.subplot2grid((1,2),(0,1))
plt.loglog(bs_avmags,'bx ')
plt.show()